In [1]:
import torch
from torchvision.models import resnet18
import torch.nn as nn
ckpt_path = 'training_checkpoints/resnet18_base_norm_padding_resize_2024-04-25 10:25:52.881424.pth'

model = resnet18()
model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
num_classes = 4
model.fc = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(model.fc.in_features, num_classes))

state_dict = torch.load(ckpt_path)

new_state_dict = {}
for key, value in state_dict.items():
    new_key = key.replace('model.', '')  
    new_state_dict[new_key] = value
model.load_state_dict(new_state_dict)

/home/adam/miniconda3/envs/cells/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>

In [2]:
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import DataLoader
import numpy as np
import torch 
from torchvision.models import resnet18
import torch.nn as nn
from dataset import ImageDataset
from albumentations import (
    Compose,
    Resize,
    OneOf,
    RandomBrightness,
    RandomContrast,
    MotionBlur,
    MedianBlur,
    GaussianBlur,
    VerticalFlip,
    HorizontalFlip,
    ShiftScaleRotate,
    Normalize,
)
mean = [0.5006, 0.3526, 0.5495]
std = [0.1493, 0.1341, 0.1124]

transform_test = Compose(
    [Normalize(mean=mean, std=std)]
)


def test_report(model, dataloader):
    """Prints confusion matrix for testing dataset
    dataloader should be of batch_size=1."""

    y_pred = []
    y_test = []
    model.eval()
    with torch.no_grad():
        for data, label in dataloader:
            output = model(data)
            label = label.numpy()
            output = output.numpy()
            y_pred.append(np.argmax(output))
            y_test.append(np.argmax(label))
        print(confusion_matrix(y_test, y_pred))
        print(classification_report(y_test, y_pred))

testset =ImageDataset(data_path='test_data',transform=transform_test)
dataloader = DataLoader(testset, batch_size=1, shuffle=True)

test_report(model.to('cpu'), dataloader)

[[697 101 195   7]
 [113 823  60   4]
 [121  51 822   6]
 [ 16  18   7  59]]
              precision    recall  f1-score   support

           0       0.74      0.70      0.72      1000
           1       0.83      0.82      0.83      1000
           2       0.76      0.82      0.79      1000
           3       0.78      0.59      0.67       100

    accuracy                           0.77      3100
   macro avg       0.77      0.73      0.75      3100
weighted avg       0.77      0.77      0.77      3100



In [3]:
def test_report(model, dataloader):
    """Prints confusion matrix for testing dataset
    dataloader should be of batch_size=1."""

    y_pred = []
    y_test = []
    model.eval()
    with torch.no_grad():
        for data, label in dataloader:
            output = model(data)
            label = label.numpy()
            output = output.numpy()
            y_pred.append(np.argmax(output))
            y_test.append(np.argmax(label))
        print(confusion_matrix(y_test, y_pred))
        print(classification_report(y_test, y_pred))
    class_report = classification_report(y_test, y_pred, output_dict=True)

    class_report = classification_report(y_test, y_pred, output_dict=True)
    
    class_keys = sorted([key for key in class_report.keys() if key.isdigit()])
    # Extract the F1-scores for the first four classes
    f1_scores = [class_report[key]['f1-score'] for key in class_keys[:4]]
    
    return tuple(f1_scores)

test_report(model, dataloader)

[[697 101 195   7]
 [113 823  60   4]
 [121  51 822   6]
 [ 16  18   7  59]]
              precision    recall  f1-score   support

           0       0.74      0.70      0.72      1000
           1       0.83      0.82      0.83      1000
           2       0.76      0.82      0.79      1000
           3       0.78      0.59      0.67       100

    accuracy                           0.77      3100
   macro avg       0.77      0.73      0.75      3100
weighted avg       0.77      0.77      0.77      3100



(0.7159732922444787,
 0.8258906171600602,
 0.7888675623800384,
 0.6704545454545454)